## Read Original Dataset mt bench, convert to datasets

In [8]:
import json
import datasets

mt_bench_en = "./datasets/final_evaluation_datasets/mt_bench.jsonl"

mt_bench_list = []
with open(mt_bench_en) as f:
    mt_bench_list = [json.loads(line) for line in f.readlines()]

mt_bench_en = datasets.Dataset.from_list(mt_bench_list)
mt_bench_en[0]

{'question_id': 81,
 'category': 'writing',
 'turns': ['Compose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experiences and must-see attractions.',
  'Rewrite your previous response. Start every sentence with the letter A.']}

In [ ]:
mt_bench_en.push_to_hub("dim/mt_bench_en")

## Evaluate original saiga

In [12]:
from datasets import load_dataset
from tqdm import tqdm

mt_bench_en = load_dataset("dim/mt_bench_en")
mt_bench_en = mt_bench_en["train"]
mt_bench_en = mt_bench_en.to_list()
mt_bench_en[0]

Found cached dataset parquet (/home/kosenko/.cache/huggingface/datasets/dim___parquet/dim--mt_bench_en-40a2837a0335d6fc/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 1/1 [00:00<00:00, 840.04it/s]


{'question_id': 81,
 'category': 'writing',
 'turns': ['Compose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experiences and must-see attractions.',
  'Rewrite your previous response. Start every sentence with the letter A.']}

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "3"
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch
from instruct_models import SaigaConversation, generate

weights_path = "IlyaGusev/saiga_7b_lora"
tokenizer_path = "IlyaGusev/saiga_7b_lora"

config = PeftConfig.from_pretrained(weights_path)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, weights_path, torch_dtype=torch.float16)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, use_fast=False)
generation_config = GenerationConfig.from_pretrained(tokenizer_path)

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/kosenko/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /home/kosenko/miniconda3/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/kosenko/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 2/2 [00:35<00:00, 17.59s/it]


### test

In [4]:
inp = "Почему трава зеленая?"
conversation = SaigaConversation()
conversation.add_user_message(inp)
prompt = conversation.get_prompt(tokenizer)

output = generate(model, tokenizer, prompt, generation_config)
print(inp)
print(output)

Почему трава зеленая?
Зеленый цвет является результатом фотосинтеза, процесса, который происходит в растениях для получения энергии из света. Когда свет попадает на хлорофилл - пигмент, который находится в клетках листьев, он преобразуется в энергию, которая используется для синтеза сахаров и других органических соединений. Этот процесс приводит к образованию зеленого цвета.


In [13]:
for i in range(len(mt_bench_en)):
    # print(item)
    item = mt_bench_en[i]
    conversation = SaigaConversation()
    mt_bench_en[i]["replies"] = []
    for turn in item["turns"]:
        print(turn)
        print("*" * 10)
        conversation.add_user_message(turn)
        prompt = conversation.get_prompt(tokenizer)

        output = generate(model, tokenizer, prompt, generation_config)
        conversation.add_bot_message(output)
        print(output)
        print("*" * 50)
        mt_bench_en[i]["replies"].append(output)
    print("=" * 100)

Compose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experiences and must-see attractions.
**********
Hawaii: A Tropical Paradise of Culture and Adventure

I recently had the opportunity to visit Hawaii for the first time, and it was truly an experience I will never forget! The islands are known for their breathtaking beaches, lush rainforests, and stunning volcanic landscapes, but what really made my trip special were all the unique cultural experiences I got to have along the way.

The first stop on my tour was Oahu, where I spent several days exploring the capital city of Honolulu. One of the most memorable things we did there was take a tour of the historic Pearl Harbor Memorial, which commemorates the Japanese attack that took place during World War II. We also visited the USS Arizona Memorial, which is built over the sunken battleship itself, and learned more about the history of the war from our guide.

After spending some time in Honolulu, w

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9 │   │   conversation.add_user_message(turn)                                                 │
│   10 │   │   prompt = conversation.get_prompt(tokenizer)                                         │
│   11 │   │                                                                                       │
│ ❱ 12 │   │   output = generate(model, tokenizer, prompt, generation_config)                      │
│   13 │   │   conversation.add_bot_message(output)                                                │
│   14 │   │   print(output)                                                                       │
│   15 │   │   print("*" * 50)                                                                     │
│                                                                                                  │
│ /cephfs/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supe │
│ rvised_finetuning/instruct_models.py:85 in generate                                              │
│                                                                                                  │
│   82 def generate(model, tokenizer, prompt, generation_config):                                  │
│   83 │   data = tokenizer(prompt, return_tensors="pt")                                           │
│   84 │   data = {k: v.to(model.device) for k, v in data.items()}                                 │
│ ❱ 85 │   output_ids = model.generate(**data, generation_config=generation_config)[0]             │
│   86 │   output_ids = output_ids[len(data["input_ids"][0]) :]                                    │
│   87 │   output = tokenizer.decode(output_ids, skip_special_tokens=True)                         │
│   88 │   return output.strip()                                                                   │
│                                                                                                  │
│ /home/kosenko/miniconda3/lib/python3.10/site-packages/peft/peft_model.py:743 in generate         │
│                                                                                                  │
│    740 │   │   │   self.base_model.generation_config = self.generation_config                    │
│    741 │   │   try:                                                                              │
│    742 │   │   │   if not isinstance(peft_config, PromptLearningConfig):                         │
│ ❱  743 │   │   │   │   outputs = self.base_model.generate(**kwargs)                              │
│    744 │   │   │   else:                                                                         │
│    745 │   │   │   │   if "input_ids" not in kwargs:                                             │
│    746 │   │   │   │   │   raise ValueError("input_ids must be provided for Peft model generati  │
│                                                                                                  │
│ /home/kosenko/miniconda3/lib/python3.10/site-packages/torch/utils/_contextlib.py:115 in          │
│ decorate_context                                                                                 │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                    